In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
%matplotlib inline

# 4.6.2 The stock market data

In [ ]:
df = sm.datasets.get_rdataset("Smarket", "ISLR", cache=True).data

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.corr().style.background_gradient(cmap='viridis')

In [ ]:
df["Volume"].plot();

# 4.6.2 Logistic Regression

In [ ]:
y = pd.get_dummies(df["Direction"])["Up"]
X = df.drop(columns=["Direction", "Today", "Year"])

In [ ]:
logit = sm.Logit(y, sm.add_constant(X)).fit()
print(logit.summary())

Access the coefficients and p values

In [ ]:
logit.params

In [ ]:
logit.pvalues

Display the first 10 predicted probabilities from the training data

In [ ]:
predict_prob = logit.predict(sm.add_constant(X))
predict_prob[:10]

Note to self, don't call fittedvalues for logistic regression, it just returns the dot product of the training exogenous variables and the coefficients.

Make a confusion matrix

In [ ]:
class_labels = ["Down", "Up"] # took the Up dummy column as my independent variable, so 1 = Up
predict_class = pd.Series(data=0, index=predict_prob.index)
predict_class.loc[predict_prob > 0.5] = 1
confusion_mat = confusion_matrix(y, predict_class)
fig, ax = plot_confusion_matrix(conf_mat=confusion_mat, class_names=class_labels)
plt.show()

All my attempts at doing a confusion matrix come out like this. I'm tempted to blame Matplotlib. None of the examples I've referenced look this bad.